In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
import numpy as np
import mlflow
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from mlflow.models.signature import infer_signature
from mlflow import MlflowClient

In [2]:
train_path = '../data/processed/train.csv'
test_path = '../data/processed/test.csv'
metrics_path ='../reports/metrics.txt'

In [3]:
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return transform_categorical(X)
 
def transform_categorical(column):
    print (column)
    vehicles_available = column['vehicles_available']
    vehicles_available = vehicles_available.apply(str).str.replace('<', '')
    nan_mask = vehicles_available.isna()
    vehicles_availablenan = vehicles_available[nan_mask]

    vehicles_available = vehicles_available.str.strip().dropna().loc[lambda x: x.str.len() > 0]
    vehicles_available = vehicles_available.astype('float')

    vehicles_availablenan[:] = vehicles_available.mean()
    vehicles_available = pd.concat([vehicles_available, vehicles_availablenan], axis=0)

    column['vehicles_available'] = vehicles_available.astype('float')
    return column

In [4]:
def transform_test(test_path, categorical_transform, numerical_transform):
    csv = pd.read_csv(test_path, encoding='utf_8')
    test = pd.DataFrame(csv)
    print('Data read')

    cat_features = test.select_dtypes(include=['object']).columns.tolist()
    num_features = test.select_dtypes(
        include=['float64', 'int64']).columns.tolist()

    test_preprocessor = ColumnTransformer([
        ('availability_transform', categorical_transform, cat_features),
        ('normal_transform', numerical_transform, num_features)
    ])

    arr = test_preprocessor.fit_transform(test)

    test = pd.DataFrame(
        arr, columns=['hour', 'vehicles_available', 'temp', 'hum', 'percp', 'wspeed', 'capacity', 'capacity_free'])
    return test.astype('float')

In [5]:
MLFLOW_TRACKING_URI = "https://dagshub.com/anagasperin/IIS_projekt.mlflow"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("mlruns")
mlflow.autolog()

2023/07/15 22:21:22 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/07/15 22:21:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [6]:
csv = pd.read_csv(train_path, encoding='utf_8')
train = pd.DataFrame(csv)

x_train = train.drop('vehicles_available', axis=1)
y_train = pd.DataFrame(train['vehicles_available'])

num_features = ['temp', 'hum', 'percp', 'wspeed', 'capacity', 'capacity_free']

categorical_transform = Pipeline([
    ('transformer', CategoricalTransformer())
])

arr = categorical_transform.fit_transform(y_train)
y_train = pd.DataFrame(arr, columns=['vehicles_available'])

numerical_transform = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer([
    ('numerical_transform', numerical_transform,
        num_features),
])

pipe = Pipeline([
    ('preprocess', preprocessor),
    ('MLPR', MLPRegressor())
])

parameter_space = {
    "MLPR__hidden_layer_sizes": [(32), (16)],
    "MLPR__learning_rate_init": [0.001, 0.01]
}

search = GridSearchCV(pipe, parameter_space,
                        verbose=2, error_score='raise')
search.fit(x_train, y_train)

test = transform_test(test_path, categorical_transform, numerical_transform)
x_test = test.drop('vehicles_available', axis=1)
y_test = pd.DataFrame(test['vehicles_available'])


signature = infer_signature(x_train, search.predict(x_test))
mlflow.sklearn.log_model(search, signature=signature, artifact_path="MLPRegressor",
        registered_model_name="MLPRegressor")

prediction = search.predict(x_test)
print('Model trained')

# Calculate MSE and MAE for the test data
mse_test = mean_squared_error(y_test, prediction)
mae_test = mean_absolute_error(y_test, prediction)
evs_test = explained_variance_score(y_test, prediction)

mlflow.log_metric("MSE Test", mse_test)
mlflow.log_metric("MAE Test", mae_test)
mlflow.log_metric("EVS Test", evs_test)

with open(metrics_path, 'w') as file:
    file.write('MAE:' + str(mae_test) + '\n')
    file.write('MSE:' + str(mse_test) + '\n')
    file.write('EVS:' + str(evs_test) + '\n')

print('Reports updated')

#mlflow.sklearn.log_model(search, "MLPRegressor")

print('Model serialized')

autolog_run = mlflow.last_active_run()
print(autolog_run)

2023/07/15 22:21:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6ebfa4f98721431bafffed9e0abc1c5a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/07/15 22:21:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these c

     vehicles_available
0                    12
1                    14
2                    11
3                    14
4                    11
..                  ...
806                   9
807                   7
808                   6
809                   6
810                   4

[811 rows x 1 columns]


2023/07/15 22:21:24 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/07/15 22:21:24 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature
2023/07/15 22:21:24 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/07/15 22:21:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/07/15 22:21:31 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[CV] END MLPR__hidden_layer_sizes=16, MLPR__learning_rate_init=0.01; total time=   0.1s
[CV] END MLPR__hidden_layer_sizes=16, MLPR__learning_rate_init=0.01; total time=   0.0s


c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
2023/07/15 22:21:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred sche

Data read


2023/07/15 22:21:50 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '08295a9639494ad59fe2dcfedfd2548b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/07/15 22:21:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Uporabnik\AppData\Local\pypoetry\Cache\virtualenvs\bike-availability-mb-RteGF7NQ-py3.10\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these c

Model trained
Reports updated
Model serialized
<ActiveRun: >


In [18]:
with mlflow.start_run():
    run = mlflow.active_run()
    run_id = run.info.run_id

    client = MlflowClient()
    m = client.get_latest_versions('MLPRegressor', stages=["Production"])
    history = client.get_metric_history(m.run_id, key='MAE Test')

    min = history[0].value
    for h in history:
        if h.value < min:
            min = h.value

    if mae_test < min:
        print('jere')
        client.transition_model_version_stage(
            name="MLPRegressor", version=m.version, stage='Production'
        )

    mlflow.end_run()

AttributeError: 'list' object has no attribute 'start_run'

In [ ]:
model_version_info = client.get_latest_versions("MLPRegressor", stages=["Production"])[0]
model_uri = model_version_info.source

# Load the model
loaded_model = mlflow.pyfunc.load_model(model_uri)

x_test['hum'] = x_test['hum'].astype('int')

# Use the model to make predictions
predictions = loaded_model.predict(x_test)
print(predictions)